In [4]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup


############ MONTHS ##############
page = requests.get("https://coronavirus.dc.gov/page/coronavirus-data")
soup = BeautifulSoup(page.content, 'html.parser')

ct = 0 
ray = np.array([])



for path in soup.find_all('p'):
    
    phrase = path.get_text() ## stores phrase and checks if keyword is in phrase
    
#     if (phrase.find('April') != -1) or (phrase.find('March') != -1): # raw months outpout
#         print (phrase)

    year = phrase.find('2020') +4 #clean months output
    if (phrase.find('April') != -1):
        pos = phrase.find('April')
#         print (phrase[pos : year])

        date = phrase[pos : year]
        if (date != ''):
            ray = np.append(ray, date)
        ct += 1
    
    elif (phrase.find('May') != -1):
        pos = phrase.find('May')
        date = phrase[pos : year]
        if (date != ''):
            ray = np.append(ray, date)
        ct += 1        

    elif (phrase.find('March') != -1):
        pos = phrase.find('March')
        date = phrase[pos : year]
        if (year != 3):
            if (date != ''):
                date = phrase[pos : year]
                ray = np.append(ray, date)
#             print (phrase[pos : year])
            ct += 1
        else:
            date = phrase[pos:len(phrase)-1]+", 2020"
            if (date != ''):
                ray = np.append(ray, date)
            if (phrase.find('March 11')):
                break




def overall (phrase,key): 
   
    colon = phrase.find(':')
    if (phrase.find(key) != -1) or (phrase.find(key.capitalize()) != -1):
       
        if (colon != -1):
#             print (phrase[colon+2:len(phrase)])
            return (phrase[colon+2:len(phrase)])

        else: 
#             print (phrase[:len(phrase)])
            return (phrase)
            
    
def positive (phrase,key, ct2):
    colon = phrase.find(':')
    if (phrase.find(key) != -1) or (phrase.find(key.capitalize()) != -1):
        
        if (phrase.find('PHL') != -1):
            ct2 +=1
#             print (ct2, phrase[colon+2:len(phrase)], end = ' ')
            x = phrase[colon+2:len(phrase)]
            return x,ct2
        
        elif (phrase.find(';') != -1):
#             print ('')
            x = ''
            return x, ct2
        
        else:        
            if (phrase.find('commercial lab') != -1):
                ct2 = 2
#             print (phrase[colon+2:len(phrase)])
            x = phrase[colon+2:len(phrase)]
            return x, ct2






ct2 = 0 

nums = np.array([])
deaths = np.array([])
recover = np.array([])
pos = np.array([])


for stat in soup.find_all('li'):
    
    total = None
    phrase = stat.get_text()
    phrase = phrase.replace(',', '')
    phrase = phrase.replace('*','')
    
    key = "overall"
    total = overall(phrase,key)
    if (total != None):
        total = total.replace(' ','')
        total = int(total)
        nums = np.append(nums, total)
    
    
    key = 'lost' 
    total = overall(phrase,key)  
    if (total != None):
        deaths = np.append(deaths, total)
    key = 'deaths' 
    total = overall(phrase,key)  
    if (total != None):
        deaths = np.append(deaths, total)
        
        
    key = 'recovered'
    total = overall(phrase,key)  
    if (total != None):
        recover = np.append(recover, total)
        
        
    key = "positives"
    total = positive(phrase,key,ct2) 

    if (total != None):
        if (total[1] == 2):
            x = len(pos) - 1
            pos[x] = int(pos[x])
#             y = int(total[0])
            pos[x]= pos[x] + ' ' + total[0]
#             print('hi')
        elif (total[0] != ''):
            pos = np.append(pos, total[0])
        

allz = np.array([])
for x in pos:
    finder = x.find(' ')
    if(finder != -1):
        first = int(x[0:finder])
        second = int (x[finder+1:len(x)])     
        total = first + second 
#         print (total)
        allz = np.append(allz, total)

    else:
        total = int(x)
        allz = np.append(allz, total)
#         print(x)

if (len(allz) != len(nums)):
    allz = np.delete(allz,len(allz)-1)
    ray = np.delete(ray,len(ray)-1)

ct = 0
# for x in ray:
#     comma = x.find(',')
#     if (x.find('March') != -1):
#         x = '3/'+ x[6:comma]
#         ray[ct] = x
#     else:
#         x = '4/'+ x[6:comma]
#         ray[ct] = x
#     ct +=1

for x in ray:
    comma = x.find(',')
    if (x.find('March') != -1):
        x = '3/'+ x[6:comma]
        ray[ct] = x
    elif (x.find('April') != -1):
        x = '4/'+ x[6:comma]
        ray[ct] = x
    else:
#         print(x)
        x = '5/'+ x[4:comma]
        ray[ct] = x
    ct +=1
    
ray = ray[::-1]#INVERTS ALL THE COLUMNS
allz = allz[::-1]
nums = nums[::-1]




In [35]:
redo = np.array([])  ### FILLS DATAFRAME
rate = np.array([])

ct = 0
for x in nums:
    first = x
    if(ct == 8):
        first = int(first)
        first = first + 1000
    redo = np.append(redo,first)
    ct +=1 

    
ct = 0
for x in redo:
    x = int(x)
    net = (allz[ct]/x)*100
    rate = np.append(rate, net)
    ct +=1
    
df = pd.DataFrame({
    "Dates":ray,
    "Positives":allz,
    "Overall":redo    
})


df.Positives[53]= 5899
df.Overall[53]= 27115

df['\u0394 +'] = df['Positives'].diff(1) # DELTA POSITIVES!!!!!!!!
df['\u0394 total'] = df['Overall'].diff(1)
df['Rate'] = round((df['\u0394 +'] / df['\u0394 total'])* 100,2)
df = df[7:]

/home/roh/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/roh/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
# # df.xs('57')['\u0394 +']=10
# # df.xs('53')['Overall']= 27115
# df2 = df.copy()
# df2.Positives[53]= 5899
# df2.Overall[53]= 27115
# df = df2
print(df)




   Dates  Positives  Overall    Δ +  Δ total   Rate
7   3/21       98.0   1055.0   21.0    482.0   4.36
8   3/22      116.0   1229.0   18.0    174.0  10.34
9   3/23      137.0   1334.0   21.0    105.0  20.00
10  3/24      183.0   1609.0   46.0    275.0  16.73
11  3/25      231.0   1858.0   48.0    249.0  19.28
12  3/26      267.0   2166.0   36.0    308.0  11.69
13  3/27      304.0   2516.0   37.0    350.0  10.57
14  3/28      342.0   2812.0   38.0    296.0  12.84
15  3/29      401.0   3085.0   59.0    273.0  21.61
16  3/30      495.0   3759.0   94.0    674.0  13.95
17  3/31      586.0   4398.0   91.0    639.0  14.24
18   4/1      653.0   5070.0   67.0    672.0   9.97
19   4/2      757.0   5584.0  104.0    514.0  20.23
20   4/3      902.0   6438.0  145.0    854.0  16.98
21   4/4      998.0   6834.0   96.0    396.0  24.24
22   4/6     1211.0   7823.0  213.0    989.0  21.54
23   4/7     1440.0   8283.0  229.0    460.0  49.78
24   4/8     1523.0   8724.0   83.0    441.0  18.82
25   4/9    

In [37]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from plotly import tools
import os

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    subplot_titles=("Positives","Rate", "\u0394 +"),
                    vertical_spacing=0.07)

fig.add_trace(go.Scatter(x = df['Dates'], y = df['Positives'], name = 'Total Positives'),row=1, col=1)

fig.add_trace(
    go.Scatter(x=df['Dates'],y = df['Rate'],
    mode='lines',
    name='Rate'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df['Dates'], y=df['\u0394 +'],
    mode='markers',
    name='\u0394 +'), row=3, col=1)

fig.update_layout(height=700, width=950)
fig.update_xaxes(tickangle= 60)
fig.write_html("/home/roh/covid/_includes/graphs.html")
# fig.show()

#####DATAFRAME
df = df.iloc[::-1]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df['Dates'], df['Positives'], df['Overall'],df['\u0394 +'], df['\u0394 total'],df['Rate']],
               fill_color='lavender',
               align='left'))
])



fig.write_html("/home/roh/covid/_includes/data.html")
# fig.show()





In [40]:
# df.to_csv('out.csv', index=False)  